# ROA Calculations, RAK, Feb. 13, 2021

## Steps for Computing Vibrational Raman Optical Activity (VROA)

1. (Geometry/Structure)  Determine the nuclear geometry at which we will compute the spectrum.  Usually, this is obtained from a geometry optimization calculation.  Such a calculation minimizes the energy with respect to displacements of the nuclear positions.  We use the final, converged, lowest-energy structure.  The geometry will be in bohr/au or Angstroms in Cartesian coordinates.  For visualization, there are many tools available.  One way to visualize a geometry is to produce an .xyz file and then open it with a program such as Spartan.  For most methods in QC, analytic first derivatives (of the energy with respect to nuclear coordinates) are available to facilitate geometry optimization steps that include computing the first derivative at each step.

2. (Hessian)  The hessian is the second derivative of the energy with respect to the nuclear coordinate positions.  The hessian is used to determine the normal modes of vibration, and the corresponding force constants and vibrational frequencies.  In many programs, computation of the hessian is called a "frequency" calculation, because you need the hessian to obtain vibrational frequencies.  The hessian in PSI4 may be output into "file15", and in CFOUR into "FCMFINAL".  For some methods in QC, analytic second derivatives are available.  However, in many cases the hessian must be computed via finite differences of first derivatives (or even of energies).  If the finite-difference step size is sufficiently small and the results are sufficiently converged, the results will be equivalent.

Aside: qauntum chemistry programs may differ in where they place the origin of mass, and how they geometrically orient a molecule in space at the beginning of a calculation.  This has no effect on physical observables, but the latter will change derivatives expressed in Cartesian coordinates.  If you want to directly compare derivatives between programs, then you must ensure that the programs use the same orientation of the molecule, or else rotate the results into the frame of reference used by the other program.  For example, if you delete the first line of file15 from a PSI4 calculation, which contains the number of atoms, the file will (within numerical noise) match FCMFINAL from a CFOUR calculation - IF any difference in the orientation of the molecule is taken into account.

3. (Dipole-moment Derivatives) To compute the infrared intensities, which is the signal strength of absorbance in the infrared region for each vibrational mode, we need the dipole-moment derivatives.  The dipole moment is the first derivative of the energy wrt an applied electric field.  (A molecule with a dipole, i.e., positive and negative ends has a non-zero first-order response to an electric field.)  Dipole-moment derivatives can be thought of in two ways:  1) the derivative of the dipole moment (for each component $\mu_x,~\mu_y,~\mathrm{and}~\mu_z$) wrt displacement of the nuclear coordinates; or 2) the mixed second derivative of the energy wrt an applied electric field ($E_x,~E_y,~\mathrm{and}~ E_z$) and the nuclear positions.  The dipole moment derivatives in PSI4 may be stored in "file17" and in CFOUR in "DIPDER".

4. (Derivatives of more obscure tensors) For VROA, we need the derivatives with respect to nuclear displacements for the following 3 quantities.  The complexity of these quantities is the reason that most QC software packages are not able to compute VROA spectra, or, if they do, they can do so with only very limited options for the electronic structure theory.  These depend on the wavelength of the radiation involved.  So for different experimental wavelengths of incoming light, the computation would need to be redone.

* dipole polarizability (the 2nd derivative of the energy wrt an applied electric field; 3x3).
* optical rotation tensor (the mixed 2nd derivative of the energy wrt an applied electric field, and an applied magnetic field; 3x3).
* electric-dipole/quadrupole polarizability; (3x9)

To get the derivative of these quantities wrt nuclear displacements, we need to compute them at +/- a small difference for each nuclear coodinate in the system.  If there are $N$ atoms in the molecule, each with xyz coordinates, then there are $3N$ nuclear Cartesian coordinates.  For our finite-difference derivatives, we will need to compute these three tensors at each of $(2)(3N)=6N$ geometries. In an earlier version, if one ran Psi4 with ```properties=['roa']```, then all the needed displaced geometries would be generated, along with subdirectories containing the necessary input files, but this system is now broken and replaced as described later.

5. Compute tensors at displaced geometries.  These can be run in parallel, coarse-grained on the same node or on different computing nodes.

6. (Compute VROA parameters.) In the original, Cartesian implementation it is at this point that psi4 would read the Hessian, determine the normal modes of vibration in the molecule, and their vibrational frequencies.  Then it would read in the tensor data from the subdirectories and compute the derivatives of tensors along the normal modes.  Then it would compute the VROA parameters.

7. (Visualization). If desired, a program such as matplotlib can be used to produce simulated spectra for the different types of VROA.

### Newer, Mode-specific Approach

Steps 1-3 stay the same.  Once we get to Step 4, the approach changes.  We use the Hessian to determine the normal modes of vibration.  We then displace +/- along only those normal modes of interest to general displaced geometries.  After the reduced number of tensor calculations is complete, then the finite-differences of the tensors are computed.  The result is now immediatetly the derivate of the tensors wrt to the selected normal modes.  And the VROA parameters are then computed specifically and only for those normal modes.

The advantages are:

* when we determine the normal modes, we project out the translations and rotations so that there are only 3N-5/6 vibrations, immediately saving us 10/12 tensor computations at displaced geometries.
* experiments will generally be focused only in a certain spectral region, and we can compute the VROA spectrum only in the wavenumber range of interest
* many vibrational modes have a negligible to small VROA signal. We can identify these in advance, say by carrying out a cheap calculation and concentrate our computational efforts on the modes that are most intense, or of greatest interest.  

This input file uses the Psi4ROA class to handle all steps of an ROA calculation, though this was developed to allow you to break up the separate steps of the calculation.  Presently, the Psi4ROA class can do the following for you on request:

1. optimize structure (with optking) or set geometry
2. compute hessian with psi4 or cfour, or just read in file15/FCMFINAL
3. compute dipole derivates by finite differences with psi4 or read cfour's DIPDER
4. generate displacement vectors for cartesian or for normal mode displacements (see ```roa/mode_scatter.py/modeVectors```).  which normal modes to include can be selected by the user in the input file.
5. generate separate input files for each displacement (code used to be in psi4 driver).
6. Run the input files (coarse threading will run multiple displacements on same node at the same time, which often works well) to compute the optical rotation tensors for each displacement.
7. Read in tensors (from text output files at present) and compute the ROA parameters.  See ```roa/scatter.py/scatter()``` (this code is similar to the C++ scatter function in ccresponse) or ```roa/mode_scatter.py/modeScatter()``` which handles vibrational mode displacments.
8. Output the vibrational and ROA parameters into both a text (similar to old output) and also a dictionary output file.

I spent a great deal of time in 2019 ensuring that all the various ways of determining these quantities (Psi4/CFOUR, analytic versus finite-difference, etc.) would give the same answer.

There is also a new "ROA Normal Mode Decomposition" that shows the most significant atom-pair contributions to the ROA signal, for a selected mode.

### How I am running ROA's now

From github, get psi-rking/optking and psi-rking/roa.  The latter contains ```roa/example_hooh.py```, a sample input file for the whole calculation ROA calculation. This input file uses the Psi4ROA class to handle all steps of an ROA calculation, though this was developed to allow you to break up the separate steps of the calculation.  Presently, the Psi4ROA class can do the following for you on request:

1. optimize structure (with optking) or set geometry
2. compute hessian with psi4 or cfour, or just read in file15/FCMFINAL
3. compute dipole derivates by finite differences with psi4 or read cfour's DIPDER
4. generate displacement vectors for cartesian or for normal mode displacements (see ```roa/mode_scatter.py/modeVectors```).  which normal modes to include can be selected by the user in the input file.
5. generate separate input files for each displacement (code used to be in psi4 driver).
6. Run the input files (coarse threading will run multiple displacements on same node at the same time, which often works well) to compute the optical rotation tensors for each displacement.
7. Read in tensors (from text output files at present) and compute the ROA parameters.  See ```roa/scatter.py/scatter()``` (this code is similar to the C++ scatter function in ccresponse) or ```roa/mode_scatter.py/modeScatter()``` which handles the VROA analysis in the case of normal mode displacments.
8. Output the vibrational and ROA parameters into both a text (similar to old output) and also a dictionary output file.

I spent a great deal of time in 2019 ensuring that all the various ways of determining these quantities (Psi4/CFOUR, analytic versus finite-difference, etc.) would give the same answer.

There is also a new "ROA Normal Mode Decomposition" that shows the most significant atom-pair contributions to the ROA signal, for a selected mode.

Limitations include:
1. the cfour is a bit hardwired
2. data for hessian, dipole derivatives, and optical rotation tensors is currently read from output files.

## Using Spectrum class in roa module

In the roa module, there is a file ```demo-spectrum.py``` that gives examples of the current capabilities.

* Read ROA parameters from dictionary output files or from psi4 output files.  
* Compare the parameters between a list of spectra, and a given reference spectra (RMS, average deviation, etc.) while omitting small values below a threshold.  There is no sophisticated mode-matching here if the frequencies are very different.  
* Plot a spectrum with Gaussian or Lorenztian line-widths.  
* Plot a dual spectrum with Raman above, and ROA below.  

![ROA plot][1]

[1]: demo-roa-plot.png